In [1]:
!pip install ultralytics --upgrade

import numpy as np
import cv2
import os
import time
import yaml
import shutil
from ultralytics import YOLO
import torch
import matplotlib.pyplot as plt
from IPython.display import display, Image

# Uncomment if Ultralytics needs installation or updating
# !pip install ultralytics --upgrade

print("Libraries imported.")

# Force PyTorch to see all available GPUs (fix for Kaggle T4 x2)
# Kaggle sometimes hides the second GPU from PyTorch
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# Set device (Ensure GPU is enabled in Kaggle settings for fast training)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")

# Check for multiple GPUs
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")
    
    # List all GPUs
    for i in range(num_gpus):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"    Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.1f} GB")
    
    # For multi-GPU training with YOLO, we'll use device=[0,1]
    if num_gpus > 1:
        DEVICE = [0, 1]  # Use both GPUs
        print(f"\n✓ Multi-GPU training enabled: Using GPUs {DEVICE}")
    else:
        DEVICE = 0
        print(f"\n✓ Single GPU training: Using GPU {DEVICE}")
        print(f"\nNOTE: If you selected T4 x2 but only see 1 GPU:")
        print("  1. Make sure you saved the notebook after changing accelerator")
        print("  2. Restart the kernel (Session → Restart Session)")
        print("  3. Or use 'Quick Save' button, then reload the page")
else:
    print("No GPU available, using CPU")

# Define the working directory for outputs
WORKING_DIR = "/kaggle/working"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mkl-fft to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-random to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-umath to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 88.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# ==================================================================
# Fix NumPy 2.x Incompatibility Issue
# ==================================================================
# Downgrade numpy to 1.x for compatibility with current matplotlib/ultralytics

!pip install -q "numpy<2"

print("✓ NumPy downgraded to 1.x for compatibility")

In [2]:
# ==================================================================
# 1.5.1 Install Dependencies for Dataset Generation
# ==================================================================

# Install required packages for dataset generation
print("Installing dependencies for dataset generation...")

# Install packages one by one to ensure they're all installed
import subprocess
import sys

packages = [
    'aiohttp',
    'cairosvg',
    'gitpython',
    'tqdm',
    'opencv-python',
    'pyyaml'
]

for package in packages:
    print(f"Installing {package}...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

print("\nDataset generation dependencies installed successfully!")

# Verify imports work
try:
    import aiohttp
    import cairosvg
    import git
    import tqdm
    import cv2
    import yaml
    print("✓ All dependencies verified and ready to use")
except ImportError as e:
    print(f"✗ Import error: {e}")
    print("Please re-run this cell or restart the kernel")

Installing dependencies for dataset generation...
Installing aiohttp...
Installing cairosvg...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
Installing gitpython...
Installing tqdm...
Installing opencv-python...
Installing pyyaml...

Dataset generation dependencies installed successfully!
✓ All dependencies verified and ready to use


In [3]:
# Copy scripts from uploaded dataset
# Update 'chess-watchtower-trainer' to match your dataset name
!cp -r /kaggle/input/dataset-scripts/* .

print("Current directory:", os.getcwd())
!ls -la

Current directory: /kaggle/working
total 52
drwxr-xr-x 3 root root  4096 Nov  1 15:16 .
drwxr-xr-x 5 root root  4096 Nov  1 15:13 ..
-rw-r--r-- 1 root root 12994 Nov  1 15:16 gen_board_data.py
-rw-r--r-- 1 root root 15111 Nov  1 15:16 gen_pieces_data.py
-rw-r--r-- 1 root root  4475 Nov  1 15:16 get_pieces.py
drwxr-xr-x 2 root root  4096 Nov  1 15:13 .virtual_documents


In [7]:
from pathlib import Path

# Run get_pieces.py
!python get_pieces.py

# Verify download
assets_dir = Path('./assets/pieces')
if assets_dir.exists():
    piece_count = len(list(assets_dir.rglob('*.png')))
    print(f"\n✓ Downloaded {piece_count} piece images")
else:
    print("⚠️ Asset download failed!")

Starting Chess.com downloads...
Clone complete. Converting SVGs to PNGs...
Found 464 SVG files across all piece themes

Converting Lichess SVGs: 100%|███████████████| 464/464 [00:02<00:00, 186.29it/s]
Lichess conversion complete. Cleaning up repo...
Cleanup complete.
Chess.com downloads complete: 468/480 successful
All assets downloaded to assets/pieces

✓ Downloaded 932 piece images


In [8]:
from pathlib import Path

# Generate piece detection data (saves clean boards for next step)
!python gen_pieces_data.py --count 5000 --save-boards

# Verify generation
pieces_data = Path('yolo_pieces')
boards_data = Path('./assets/boards')

if pieces_data.exists():
    train_imgs = len(list((pieces_data / 'train' / 'images').glob('*.png')))
    val_imgs = len(list((pieces_data / 'val' / 'images').glob('*.png')))
    print(f"\n✓ Piece detection data: {train_imgs} train, {val_imgs} val")

if boards_data.exists():
    board_count = len(list(boards_data.glob('*.png')))
    print(f"✓ Clean boards saved: {board_count}")


Generating 5000 positive samples (boards with pieces)
Generating 882 negative samples (empty/sparse boards)
Total samples: 5882
Found 38 piece sets: alpha, anarcandy, caliente, california, cardinal...
Will save clean boards to assets/boards

Generating samples...
Train: 30 piece sets
Val: 8 piece sets

Train: 4000 positive + 705 negative = 4705 total
Val: 1000 positive + 177 negative = 1177 total
Generating val negative: 100%|████████████████| 177/177 [00:02<00:00, 65.54it/s]
Converting to YOLO format...
  Train: 4705 images
  Val: 1177 images
Processing val: 100%|██████████████████████| 1177/1177 [00:07<00:00, 149.30it/s]

YOLO dataset created at yolo_pieces
Train: 4705 images
Val: 1177 images

✓ Piece detection data: 4705 train, 1177 val
✓ Clean boards saved: 5882


In [9]:
from pathlib import Path

# Generate board detection data
!python gen_board_data.py --count 5000

# Verify generation
board_data = Path('yolo_board')

if board_data.exists():
    train_imgs = len(list((board_data / 'train' / 'images').glob('*.png')))
    val_imgs = len(list((board_data / 'val' / 'images').glob('*.png')))
    print(f"\n✓ Board detection data: {train_imgs} train, {val_imgs} val")

Generating 500 background images...
Generating backgrounds: 100%|█████████████████| 500/500 [00:18<00:00, 27.19it/s]
Found 5882 board images
Found 500 background images

Generating 5000 training samples...
Train: 4705 boards, 400 backgrounds
Val: 1177 boards, 100 backgrounds
Generating val samples: 100%|███████████████| 1000/1000 [00:23<00:00, 42.03it/s]

Saving YOLO format...
  Train: 4000 images
  Val: 1000 images
Processing val: 100%|███████████████████████| 1000/1000 [00:15<00:00, 65.71it/s]

YOLO dataset created at yolo_board
Train: 4000 images
Val: 1000 images

✓ Board detection data: 4000 train, 1000 val


In [10]:
# ==================================================================
# 4. Training Functions
# ==================================================================

def train_model(config_yaml, project_name, base_model='yolov8n.pt', epochs=50, imgsz=640):
    # Load the specified pretrained model
    try:
        model = YOLO(base_model) 
    except Exception as e:
        print(f"Error loading base model {base_model}: {e}")
        return None
        
    print(f"\n--- Starting Training for {project_name} using {base_model} ---")
    
    # Determine device info
    if isinstance(DEVICE, list):
        print(f"Multi-GPU Training: Using GPUs {DEVICE}")
        device_str = ','.join(map(str, DEVICE))
    else:
        device_str = str(DEVICE)
        if DEVICE == 'cuda' or (isinstance(DEVICE, int) and DEVICE >= 0):
            print(f"Single GPU Training: Using {device_str}")
        else:
            print(f"CPU Training")
    
    # Adjust batch size dynamically based on model size and number of GPUs
    # With 2 GPUs, we can use larger batch sizes
    num_gpus = len(DEVICE) if isinstance(DEVICE, list) else 1
    
    if base_model == 'yolov8n.pt':
        batch_size = 32 * num_gpus  # Scale batch size with GPUs
    else:  # yolov8s or larger
        batch_size = 16 * num_gpus
    
    print(f"Using batch size: {batch_size} (across {num_gpus} GPU(s))")

    try:
        results = model.train(
            data=os.path.join(WORKING_DIR, config_yaml), 
            epochs=epochs, 
            imgsz=imgsz, 
            device=device_str,  # Pass device as string
            patience=20, # Stop early if validation stops improving
            batch=batch_size, 
            augment=True,
            # Key Augmentations for Chess:
            perspective=0.0005, # Crucial for simulating camera angles
            degrees=15.0,
            translate=0.1,
            hsv_v=0.5, # Color/brightness augmentation
            hsv_s=0.7,
            project=project_name,
            name='train'
        )
    except Exception as e:
        print(f"An error occurred during training: {e}")
        return None
        
    # Save the best model to the working directory for easy download
    best_model_path = os.path.join(project_name, 'train/weights/best.pt')
    final_path = os.path.join(WORKING_DIR, f'{project_name}_best.pt')
    
    if os.path.exists(best_model_path):
        shutil.copy(best_model_path, final_path)
        print(f"Saved best model to {final_path}")
        
        # Display results graph
        try:
            display(Image(filename=os.path.join(project_name, 'train/results.png')))
        except: pass
            
        return final_path
    
    print(f"Best model weights not found at {best_model_path}")
    return None

In [11]:
# ==================================================================
# 4.1 Execute Training (Uncomment to Run)
# ==================================================================

# Model 1: Board Detector using YOLOv8 Nano
MODEL1_PATH = train_model('yolo_board/data.yaml', 'BoardDetector_Nano', base_model='yolov8n.pt', epochs=50, imgsz=640)

# Model 2: Piece Detector using YOLOv8 Small
MODEL2_PATH = train_model('yolo_pieces/data.yaml', 'PieceDetector_Small', base_model='yolov8s.pt', epochs=100, imgsz=640)


--- Starting Training for BoardDetector_Nano using yolov8n.pt ---
Multi-GPU Training: Using GPUs [0, 1]
Using batch size: 64 (across 2 GPU(s))
Ultralytics 8.3.223 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo_board/data.yaml, degrees=15.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.5, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=30


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/root/.config/Ultralytics/DDP/_temp_m4f4ku42140027891938448.py", line 11, in <module>
    trainer = DetectionTrainer(cfg=cfg, overrides=overrides)
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/models/yolo/detect/train.py", line 65, in __init__
    super().__init__(cfg, overrides, _callbacks)
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/trainer.py", line 160, in __init__
    self.data = self.get_dataset()
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/trainer.p

An error occurred during training: Command '['/usr/bin/python3', '-m', 'torch.distributed.run', '--nproc_per_node', '2', '--master_port', '44363', '/root/.config/Ultralytics/DDP/_temp_m4f4ku42140027891938448.py']' returned non-zero exit status 1.

--- Starting Training for PieceDetector_Small using yolov8s.pt ---
Multi-GPU Training: Using GPUs [0, 1]
Using batch size: 32 (across 2 GPU(s))
Ultralytics 8.3.223 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo_pieces/data.yaml, degrees=15.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, 


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/root/.config/Ultralytics/DDP/_temp__doc3_ci140027885499472.py", line 11, in <module>
    trainer = DetectionTrainer(cfg=cfg, overrides=overrides)
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/models/yolo/detect/train.py", line 65, in __init__
    super().__init__(cfg, overrides, _callbacks)
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/trainer.py", line 160, in __init__
    self.data = self.get_dataset()
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/trainer.p

An error occurred during training: Command '['/usr/bin/python3', '-m', 'torch.distributed.run', '--nproc_per_node', '2', '--master_port', '49125', '/root/.config/Ultralytics/DDP/_temp__doc3_ci140027885499472.py']' returned non-zero exit status 1.


## 5. Inference Pipeline

This section loads the trained models and defines the two-stage inference process, including the crucial post-processing steps of cropping (Stage 1) and coordinate mapping (Stage 2).

In [6]:
# ==================================================================
# 5. Inference Pipeline Setup
# ==================================================================

# Load the trained models.
# If you ran the training above, the paths are set automatically.
# If you uploaded pre-trained weights to Kaggle, update the paths manually here:

try:
    # Check if variables exist (set during training)
    _ = MODEL1_PATH
    _ = MODEL2_PATH
except NameError:
    # If not, define default paths (e.g., assuming weights are in the working directory)
    print("Model paths not set by training cell. Attempting to use default working directory paths.")
    MODEL1_PATH = "/kaggle/working/BoardDetector_Nano_best.pt"
    MODEL2_PATH = "/kaggle/working/PieceDetector_Small_best.pt"

model_board = None
model_pieces = None

try:
    if MODEL1_PATH and os.path.exists(MODEL1_PATH):
        model_board = YOLO(MODEL1_PATH).to(DEVICE)
        print(f"Board model loaded: {MODEL1_PATH}")
    else:
        print(f"Board model not found at {MODEL1_PATH}. Ensure training was run or path is correct.")
        
    if MODEL2_PATH and os.path.exists(MODEL2_PATH):
        model_pieces = YOLO(MODEL2_PATH).to(DEVICE)
        print(f"Pieces model loaded: {MODEL2_PATH}")
    else:
        print(f"Pieces model not found at {MODEL2_PATH}. Ensure training was run or path is correct.")

except Exception as e:
    print(f"\nError loading models: {e}")


In [7]:
# ==================================================================
# 5.1 Inference Function
# ==================================================================

def run_two_stage_inference(image_path, board_conf=0.7, piece_conf=0.4, use_clahe=False):
    """
    Implements the fast and accurate two-stage detection pipeline.
    """
    if model_board is None or model_pieces is None:
        print("Models are not loaded. Cannot run inference.")
        return None, 0, None

    start_time = time.time()

    # Load Image using OpenCV
    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        print(f"Error loading image: {image_path}")
        return None, 0, None

    # STAGE 1: Board Detection (YOLOv8 Nano)
    # Inference is fast, especially on GPU
    results_board = model_board(img_bgr, conf=board_conf, iou=0.5, verbose=False)

    if not results_board or len(results_board[0].boxes) == 0:
        # print("No board detected.")
        return None, time.time() - start_time, None

    # Post-processing Stage 1: Cropping
    # Get the highest confidence detection (YOLO sorts them)
    best_board_box = results_board[0].boxes[0].xyxy[0].cpu().numpy().astype(int)
    x1, y1, x2, y2 = best_board_box

    # Crop the image
    img_cropped = img_bgr[y1:y2, x1:x2]

    # STAGE 2: Piece Detection (YOLOv8 Small)
    # Pre-processing Stage 2: Optional CLAHE
    if use_clahe:
        img_cropped = apply_clahe(img_cropped)

    results_pieces = model_pieces(img_cropped, conf=piece_conf, iou=0.4, verbose=False)

    if not results_pieces:
        return [], time.time() - start_time, best_board_box

    # Post-processing Stage 2: Coordinate Mapping
    # Crucial Step: Map coordinates from the cropped image back to the original image space.
    final_detections = []
    # Access class names dictionary
    class_names = model_pieces.names

    for box in results_pieces[0].boxes:
        # Coordinates relative to the cropped image
        cx1, cy1, cx2, cy2 = box.xyxy[0].cpu().numpy().astype(int)
        conf = box.conf[0].item()
        cls_id = int(box.cls[0].item())

        # Map back to original image by adding the top-left corner of the board crop (x1, y1)
        orig_x1 = cx1 + x1
        orig_y1 = cy1 + y1
        orig_x2 = cx2 + x1
        orig_y2 = cy2 + y1

        final_detections.append({
            "class_name": class_names[cls_id], # FEN Notation
            "bbox": [orig_x1, orig_y1, orig_x2, orig_y2],
            "confidence": conf
        })

    end_time = time.time()
    total_time = end_time - start_time

    return final_detections, total_time, best_board_box

## 6. Visualization and Testing

In [8]:
# ==================================================================
# 6. Visualization Function
# ==================================================================

def visualize_results(image_path, detections, board_box, inference_time):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Draw board box (Green)
    if board_box is not None:
        cv2.rectangle(img, (board_box[0], board_box[1]), (board_box[2], board_box[3]), (0, 255, 0), 4)

    # Draw piece boxes
    if detections:
        for det in detections:
            bbox = det['bbox']
            label = f"{det['class_name']} {det['confidence']:.2f}"
    
            # Define colors based on FEN case (Uppercase=White, Lowercase=Black)
            if det['class_name'].isupper():
                color = (135, 206, 250) # Light Blue (for White Pieces)
            else:
                color = (255, 0, 0) # Red (for Black Pieces)
    
            cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, 2)
            
            # Add background for text readability
            (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
            cv2.rectangle(img, (bbox[0], bbox[1] - 20), (bbox[0] + w, bbox[1]), color, -1)
            text_color = (0,0,0) # Black text
            cv2.putText(img, label, (bbox[0], bbox[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 2)

    plt.figure(figsize=(12, 12))
    plt.imshow(img)
    title_text = f"Inference Time: {inference_time:.4f} seconds"
    if inference_time < 0.5:
        title_text += " (Speed Goal Met!)"
    plt.title(title_text)
    plt.axis('off')
    plt.show()

In [9]:
# ==================================================================
# 6.1 Example Execution (Test the pipeline)
# ==================================================================

# --- Example Execution ---
# Update the TEST_IMAGE_PATH to point to a specific validation image in your dataset

# try:
    # # Example path (you must change this to a real image file in your validation set)
    # # It's often useful to test on an image from the board validation set.
    # TEST_IMAGE_PATH = f"{BOARD_DATA_PATH}/val/images/example_test_image.jpg"

    # # Run inference (We recommend trying use_clahe=True to see if it improves results)
    # if os.path.exists(TEST_IMAGE_PATH):
    #     detections, inference_time, board_box = run_two_stage_inference(TEST_IMAGE_PATH, use_clahe=True)
    #
    #     if detections is not None:
    #         visualize_results(TEST_IMAGE_PATH, detections, board_box, inference_time)
    # else:
    #      print(f"Test image not found at: {TEST_IMAGE_PATH}. Please update the path.")
# except NameError:
    # print("Skipping test execution because data paths (like BOARD_DATA_PATH) might not be properly defined yet.")
